## Recommender system - Collaborative filtering from scratch( No use of Deep learning library or Scikit learn)

In [ ]:
import numpy as np
import pandas as pd
from resources import *

In [ ]:
R_file = 'data\small_movies_R.csv'
W_file = 'data\small_movies_W.csv'
X_file = 'data\small_movies_X.csv'
Y_file = 'data\small_movies_Y.csv'
b_file = 'data\small_movies_b.csv'


#### Import data and Define varaible 

In [ ]:
R = import_movies_data(R_file)
Y = import_movies_data(Y_file)
movie_list = import_movie_list()
movies = movie_list['title']
movies = np.array(movies).reshape(-1,1)

num_movies = R.shape[0]

#### Create a new user , for which we will recommend the movies

In [ ]:
new_user = np.zeros((Y.shape[0],1))
# Design something to take input from user 

# say = "continue"
# while say.lower() != "exit" :
#     a = int(input("Enter the Movie ID :  "))
#     while a > Y.shape[0] or a < 0 :
#         a = int(input("Enter the Movie ID which exist int real : "))
    
#     b = float(input(f"Enter the rating you give to {movies[a]} afterwatching it : "))
#     while b > 5 or b < 0: 
#         b = float(input(f"Enter a rating between 0 to 5 ,  it can be decimal too ! : "))

#     new_user[a] = b 
#     say = input("type 'exit' to stop writing the movies otherwise write 'con' to continue typing movies")

new_user = np.zeros((num_movies,1))
new_user[62] = 1
new_user[2716] = 5
new_user[3618] = 1
new_user[1398] = 5
new_user[3083] = 5
new_user[2112] = 3    
new_user[2755] = 4
new_user[4170] = 4
new_user[378] = 5
new_user[3773] = 3
new_user[4415] = 4
new_user[3009] = 1
new_user[1088] = 1
new_user[3556] = 5

my_rated = [i for i in range(num_movies) if new_user[i] > 0 ]
for i in range(num_movies):
  if(new_user[i] > 0):
    print(f"User rated {float(new_user[i])} to movie : {movies[i]}")


#### Add the new user to the Actual dataset ,  New user is added at the start of the dataset

In [ ]:
# movies[new_user != 0] , new_user[new_user != 0]
Y = np.c_[new_user , Y]
R = np.c_[(new_user != 0).astype(int), R]
Y.shape , R.shape

In [ ]:
Y_norm , Y_mean = mean_normalize_data2(Y , R)
num_users = R.shape[1]
num_genre = 15

In [ ]:
M = Y.shape[0]
U = Y.shape[1]
X = np.random.randn(M, num_genre)  
W = np.random.randn(U, num_genre)  
b = np.random.randn(1, U) 
Y = Y_norm

#### Define the derivative of Cost function J with respect to the W(user/genre)

In [ ]:
def dJ_dw(W,X,R,Y,b,lambda__):
    ans = 2*np.dot( ((np.dot(X,W.T) + b - Y).T)*R.T ,X) + lambda__*W
    return ans  

#### Define the derivative of Cost function J with respect to the X(movies/genre)

In [ ]:
def dJ_dX(W,X,R,Y,b,lambda__):
    ans = 2*np.dot( (np.dot(X,W.T) + b - Y )*R, W) + lambda__*X
    return ans


#### Define the derivative of Cost function J with respect to the b(user)

In [ ]:
def dJ_db(W,X,R,Y,b):
    ans = 2*np.sum( ( np.dot(X,W.T) + b - Y )*R ,axis=0)
    return ans

#### Define the cost function

In [ ]:
def cost_fucntion(W,X,Y,R,b,lambda__):
    cost = 0
    cost1 = 1/2*np.square(np.sum(np.dot(X,W.T)*R + b*R - Y*R)) 
    cost2 = lambda__*np.sum(W*W) 
    cost3 = lambda__*np.sum(X*X)
    return cost1 + cost2 + cost3                      

#### Iterate the model using defined learning rate and regularization constant

In [ ]:
iterations = 2000
alpha = 1e-4
lambda__ = 1
cost_old  = cost_fucntion(W,X,Y,R,b,lambda__)
for i in range(iterations):
    W = W - alpha*dJ_dw(W,X,R,Y,b,lambda__)
    X = X - alpha*dJ_dX(W,X,R,Y,b,lambda__)
    b = b - alpha*dJ_db(W,X,R,Y,b)
    cost = cost_fucntion(W,X,Y,R,b,lambda__)
    print(f"Cost function value after the iteration {i} is : {cost}")
    # put some else condition or something else to make it fast

#### predict the data from the learned model 

In [ ]:
predictions_normalised =  np.matmul(X,W.T) + b
prediction = predictions_normalised + Y_mean
pred = prediction[:,0]

#### Check the predicted rating of the movies user had already rated

In [ ]:
for i in range(len(new_user)):
    if new_user[i] > 0:
        print(f'Original {new_user[i]}, Predicted {pred[i]:0.2f} for {movies[i]}')

#### Return the top 30 highly rated movie for new user

In [ ]:
ix = np.argsort(pred)
for i in range(len(pred)-1-30 , len(pred)):
    j = ix[i]
    if j not in my_rated:
        print(f'Predicting rating {pred[j]:0.2f} for movie {movies[j]}')